In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# Define the URL
url = "https://www.upwork.com/freelance-jobs/python-script/"
try:
    # Sending HTTP request to the given URL
    response = requests.get(url)
    response.raise_for_status()  # Trigger HTTPError for 4xx/5xx status codes
    print("Connected successfully")
except requests.exceptions.HTTPError as errh:
    print(f"HTTP Error: {errh}")
except requests.exceptions.ConnectionError as errc:
    print(f"Error Connecting: {errc}")
except requests.exceptions.Timeout as errt:
    print(f"Timeout Error: {errt}")
except requests.exceptions.RequestException as err:
    print(f"Request Error: {err}")
else:
    # Parsing the content of the response
    content = response.text
    soup = BeautifulSoup(content, 'lxml')

    # Mengambil semua elemen dengan kelas 'job-tile-wrapper'
    job_tiles = soup.find_all('div', class_='job-tile-wrapper')

    # List to store the job data
    jobs_data = []

    for job_tile in job_tiles:
        # Finding the job title
        job = job_tile.find('a', class_='up-n-link d-block display-u2u job-title h5 mb-3x')
        job_title = job.text.strip() if job else "N/A"

        # Finding the posting time
        post_time = job_tile.find('small', class_='text-muted-on-inverse')
        post_time_detail = post_time.find_next('small', class_='text-muted-on-inverse').text.strip() if post_time else "N/A"
        post_time_cleaned = post_time_detail.replace('-', '  ')

        # Finding the job level
        level = job_tile.find('p', class_='span-6 span-md-4 mb-0 pb-4x pb-md-6x')
        level_detail = level.find('strong').text.strip() if level else "N/A"

        # Finding the duration
        duration = job_tile.find('p', class_='span-6 span-md-4 mb-0 pb-4x pb-md-6x pr-0')
        duration_detail = duration.text.strip() if duration else "N/A"

        # Append the data to the list
        jobs_data.append({
            "Job Title": job_title,
            "Time Posted": post_time_cleaned,
            "Level": level_detail,
            "Duration": duration_detail
        })

    # Create a DataFrame from the list
    df = pd.DataFrame(jobs_data)

    # Remove rows with 'N/A' in all columns
    df.replace("N/A", pd.NA, inplace=True)
    df.dropna(how='all', inplace=True)

    # Save DataFrame to Excel file
    file_name = "job_listings.xlsx"
    df.to_excel(file_name, index=False, engine='openpyxl')

    print(f"Data has been saved to {file_name}")